# Bikes 🚲

[Lecture slides](https://docs.google.com/presentation/d/1lQ3AcIMXd1pNFiAroYExzLOLq2HGSWrJj0p8OL8vT8Q/edit#slide=id.p22)

----

The data from today is about bike sharing demand from [a previously held Kaggle competition](https://www.kaggle.com/c/bike-sharing-demand/data).  Below is a bit of documentation from the competition.

---

> You are provided hourly rental data spanning two years. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period.
>
> Data Fields
> ```
> datetime - hourly date + timestamp  
> season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
> holiday - whether the day is considered a holiday
> workingday - whether the day is neither a weekend nor holiday
> weather - 1: Clear, 
>              Few clouds,
>              Partly cloudy, 
>              Partly cloudy
>           2: Mist + Cloudy, 
>              Mist + Broken clouds, 
>              Mist + Few clouds,
>              Mist
>           3: Light Snow,
>              Light Rain + Thunderstorm + Scattered clouds, 
>              Light Rain + Scattered clouds
>           4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, 
>              Snow + Fog 
> temp - temperature in Celsius
> atemp - "feels like" temperature in Celsius
> humidity - relative humidity
> windspeed - wind speed
> casual - number of non-registered user rentals initiated
> registered - number of registered user rentals initiated
> count - number of total rentals
> ```

From this list of column descriptions.

* What columns are categorical?  Which of these do you think we'll need to one-hot-encode?
* Do you see any columns that you expect to be multicollinear?

In the Kaggle competition the challenge was to predict the `count` column.  The `casual` and `registered` columns aren't allowed to be used since that information is really just the `count` column broken out into its components (i.e. `count = casual + registered`).

In [ ]:
%load_ext nb_black

In [ ]:
import warnings

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")


def one_hot_encode(X, encode_cols, index=None):
    other_cols = [c for c in X.columns if c not in encode_cols]

    ct = ColumnTransformer(
        #   Format
        #   [("name of step", what_to_do(), [what columns to do it to])]
        [("one hot encode", OneHotEncoder(drop="first", sparse=False), encode_cols)],
        remainder="passthrough",
    )

    ct.fit(X)

    # This is not the flexible and definitely not the most
    # readable way to get column names, a function would be better
    encoded_names = ct.transformers_[0][1].get_feature_names()
    encoded_names = list(encoded_names)

    X_encoded = ct.transform(X)
    X_encoded = pd.DataFrame(X_encoded, columns=encoded_names + other_cols, index=index)

    for i, name in enumerate(encode_cols):
        X_encoded.columns = X_encoded.columns.str.replace(f"^x{i}", name)

    return X_encoded


def eval_preds(y_true, y_pred):
    error = y_true - y_pred

    rmse = np.sqrt((error ** 2).mean())
    mae = error.abs().mean()
    mape = (error / y_true).abs().mean()

    print(f"rmse {rmse:.2f}")
    print(f"mae {mae:.2f}")
    print(f"mape {mape:.2f}")

    line_pts = [y_true.min(), y_true.max()]
    plt.scatter(y_true, y_pred)
    plt.plot(line_pts, line_pts, c="red", ls="--", alpha=0.5)
    plt.xlabel("Actual")
    plt.ylabel("Fit")
    plt.show()

### Let's do some general EDA

In [ ]:
# data from https://www.kaggle.com/c/bike-sharing-demand/data
data_url = "https://docs.google.com/spreadsheets/d/1GJrx_Y3cvD1sWLg_zF_mMZZ0iZ_If2E02VIkyCi3PTc/export?format=csv"
bike_sharing = pd.read_csv(data_url)
bike_sharing.shape

The column `datetime` itself isn't very useful.  Why is that and how might we get some useful info out of the column?

Drop the datetime column

Investigate the column datatypes to make sure there's nothing unexpected

Do we have missing values we need to handle?

Calculate summary statistics.  When doing this, pay extra attention to our response variable, `count`.

Plot a `scatter_matrix`/`pairplot` of our dataframe

Some things to point out.

* Its not the strongest thing, but, temperature seems to have an effect on rentals (the higher the temp the more rentals).  This effect seems more prominent in the casual rentals.
* Both humidity and windspeed seem to have negative effects.  When they're low, there doesn't seem to be a big correlation, but when these factors are high we see a drop off in rentals.
* We won't be using this information, but note the plots between casual & count.  We see a pretty distinct v-shape, this is indicative of an interaction happening somewhere.  That is, there seems to be 2 distinct groups of data, we could try and explore and find a reason for this split.
* Our response variable `count` is a very positively skewed distribution.  This indicates that we might want to perform a transformation to it.  For now we'll leave it, and come back to this.

Plot the distribution of the response

In [ ]:
plt.hist(bike_sharing["count"])
plt.show()

The boxplot of our response variable also shows some outliers we might want to deal with.  This is something we might come back to.

## Prep data for modeling

One-hot encode features where appropriate

In [ ]:
cat_columns = ["season", "weather", "weekday", "hour"]


Separate data into its `X` and `y` components, and perform a `train_test_split`

Assess multicollinearity with variance inflation factor.

Address any issues with collinearity

## Build a ridge regression model

Build a plain ole `LinearRegression` model for comparison

Build a fancy ole Ridge regression model

In [ ]:
# See documentation of ridge regression to see that score here is R^2
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
print(f"LinearRegression Train R^2: {linear.score(X_train, y_train):.2f}")
print(f"LinearRegression Test R^2: {linear.score(X_test, y_test):.2f}")

print(f"\nRidge Train R^2: {ridge.score(X_train, y_train):.2f}")
print(f"Ridge Test R^2: {ridge.score(X_test, y_test):.2f}")

No real change in our $R^2$ here.. whats the deal man? I thought this was used to make things better.  Remember what the loss function is doing.  We're including a penalty for large coefficients, so let's see how they changed.

In [ ]:
print(np.sum(linear.coef_ ** 2))
print(np.sum(ridge.coef_ ** 2))

print(linear.coef_)
print(ridge.coef_)

Let's actually see numbers other than $R^2$.  How are our predictions?

In [ ]:
y_pred_train = ridge.predict(X_train)
y_pred_test = ridge.predict(X_test)

print("Train\n---------------------------------")
eval_preds(y_train, y_pred_train)
print("Test\n---------------------------------")
eval_preds(y_test, y_pred_test)

We want to fall evenly around that line.  Right now, we're underpredicting high values.  In our EDA we made a comment about our response's skewed distribution.  We might want to reconsider a transformation.  A log transformation is a good one to start with for right skewed data.

Refit the model on a logged version of `y`.

Re-evaluate the predictions

In [ ]:
y_pred_train = _____
y_pred_test = _____

print("Train\n---------------------------------")
print(f"R^2: {ridge.score(X_train, np.log(y_train)):.2f}")
eval_preds(y_train, y_pred_train)
print("Test\n---------------------------------")
print(f"R^2: {ridge.score(X_test, np.log(y_test)):.2f}")
eval_preds(y_test, y_pred_test)

## Build a LASSO regression model 🤠

How are our coefficients looking now?

Well... the coefficient shrinking definitely did its thing here... We probably want a better way of selecting an alpha than guess and check.  This is where cross validation comes into play paired with something called a gridsearch.  All the grid search does is find the best combination of hyperparameters that we tell it to test.  Here our only hyperparameter will be alpha.

Let's fit lasso using a gridsearch for the best alpha.  After we fit the model lets view the best alpha and the new coefficients.

In [ ]:
grid = ____

lasso_cv = GridSearchCV(Lasso(), grid, verbose=1)
lasso_cv

# The best fit is in the best_estimator_ attribute
print(f"selected alpha: {lasso_cv.best_estimator_.alpha}")
lasso_cv.best_estimator_.coef_

Evaluate the predictions

In [ ]:
y_pred_train = ____
y_pred_test = ____

print("Train\n---------------------------------")
print(f"R^2: {lasso_cv.score(X_train, np.log(y_train)):.2f}")
eval_preds(y_train, y_pred_train)
print("Test\n---------------------------------")
print(f"R^2: {lasso_cv.score(X_test, np.log(y_test)):.2f}")
eval_preds(y_test, y_pred_test)

We're now on par with the ridge regression results.  Not the most fair comparison though, with ridge regression we didn't grid search, we just made up a value for alpha.  On your own you can implement grid search for the ridge regression and see how/if the results improve.

## Build an elasticnet regression model

In [ ]:
grid = ____
elasticnet_cv = GridSearchCV(ElasticNet(), grid, verbose=1)
elasticnet_cv.fit(X_train, np.log(y_train))

print(f"selected alpha: {elasticnet_cv.best_estimator_.alpha}")
print(f"selected l1_ratio: {elasticnet_cv.best_estimator_.l1_ratio}")
elasticnet_cv.best_estimator_.coef_

The l1_ratio selected was 1.0... what does this mean?  (the answer to this question is also the reason why there's no analysis on the model results)